In [234]:
from collections import Counter
import re
from PIL import Image, ImageEnhance


def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    # print('Texts:')
    list_t = []
    for text in texts:
        # try:
        #     text = float(text.description))
        # except:
        text = text.description
        list_t.append(text)
        #print('\n"{}"'.format(text.description))

        # vertices = (['({},{})'.format(vertex.x, vertex.y)
        #             for vertex in text.bounding_poly.vertices])

        # print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return list_t

def islen(num):
    try:
        if(float(num) < 1000) and len(num) > 1:
            return True
    except ValueError:
        return False
    
def check_num(num):
    try:
        float(num)
        return True
    except:
        return False

def replace_invalid(list_detected_text_2):
    f = lambda x: x.replace(" ", ",", ":", ";", "!", "-", "=", "+")
    invalid_chars = [' ', ',', ';', '{', '}', '(', ')', '\n', '\t', '=', ":", ";", "!", "-", "=", "+","/"]
    invalid_table = str.maketrans({k:'' for k in invalid_chars})
    list_valid = [a.translate(invalid_table) for a in list_detected_text_2]
    return list_valid

def check_point(x,i):
    try:
        x.index('.') == i
        return True
    except: 
        return False

def check_point2(x):
    try:
        x.index('.')
        return True
    except:
        return False
def check_len(str1):
    try:
        if float(str1) > 100000:
            return True
        else:
            return False
    except:
        return False

def last_check(img):
    temp = 0
    for factor in [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0, 2.1]:
        a = detect_cordinate2(img, factor)
        
        if list(a[0].values()).count(list(a[0].values())[0]) == 2:
            break;
        else:
            temp = factor + 0.5
    if factor == 2.1:
        a1 = detect_cordinate2(img, 1)
        return False, a1, 1
    else: 
        return True, a, factor
        
def detect_cordinate2(img, p_factor):
    img2 = enhence_resolution(img, p_factor)
    list_t_f = detect_text(img2)
    list_temp = list_t_f[0].split('\n')

    list_temp11 = []
    for a in list_temp:
        if (" " in a) and (len(a) > 12):
            list_temp11 = list_temp11 + a.split(" ")
        else:
            list_temp11.append(re.sub(' +', '.', a))

    list_temp1 = replace_invalid(list_temp11)

    list_temp2 = []
    for a in list_temp1:
        if a[0:1].isdigit():
            # a1 = a.replace(' ','.')

            a1 = re.sub(' +', '.', a)
            a2 = a1.replace(',','.')
            list_temp2.append(a2)
            
    list_fre = [a[:2] for a in list_temp2]
    c = Counter(list_fre)
    list_com = [a[0] for a in c.most_common(2)]
    list_com.sort()
    

    # index_point = []
    # for a in list_temp2:
    #     try:
    #         index_point.append(a.index('.'))
    #     except:
    #         index_point.append(-1)

    
    
    new_list_temp = []   
    for l in list_temp2:
        if check_point2(l) == False:
            if l[:2] == list_com[0]:
                i = 7
            else:
                i = 6
            new_list_temp.append(l[:i]+"."+l[i:])
        else:
            new_list_temp.append(l)
    
#     list_fre = [a[:2] for a in new_list_temp]
#     c = Counter(list_fre)
#     list_com = [a[0] for a in c.most_common(2)]
#     list_com.sort()
    
    dict_temp = {}
    count = 0
    i= 7
    for a in list_com:
        dict_temp[count] = [re.findall("\d*.\d+", b)[0].replace('-','') for b in new_list_temp if b.startswith(a) and (check_len(b) == True) and ("12345" not in b)]
        # temp = []
        # for a in dict_temp[count]:
        #     if check_point(a,i) == True:
        #         temp.append(a)
        #     else:
        #         temp.append(a[:i]+"."+a[i:])
        # dict_temp[count] = temp
        # i = 6
        count = count + 1
    
    cor = {}
    count = 1
    for i,j in zip(dict_temp[0],dict_temp[1]):
        cor[count] = (float(i),float(j))
        count = count + 1
    
    
    list_len =  [float(a) for a in new_list_temp[-len(cor):] if islen(a)]
    
    return cor, list_len

def enhence_resolution(img, p_factor):
    im = Image.open(img)
    enhancer = ImageEnhance.Contrast(im)
    factor = p_factor #gives original image
    im_output = enhancer.enhance(factor)
    name = 'original-image.png'
    im_output.save(name)
    return name

In [ ]:
# a1 = detect_cordinate2("images/OCR_Internet_Collection_EUS30.jpg", 2)
# c = Counter(a[0].values())
# a1

In [235]:
last_check("images/OCR_Internet_Collection_EUS17.jpg")
# list(a[0].values()).count(list(a[0].values())[0])

(False,
 ({1: (1197374.41, 586741.99),
   2: (1197374.33, 588736.99),
   3: (1197397.33, 588737.53),
   4: (1197397.15, 588741.99)},
  [5.0, 23.01, 5.0, 22.75]),
 1)

In [221]:
# # a = enhence_resolution("images/OCR_Internet_Collection_EUS33.jpg", 1)
# a = detect_cordinate2("images/OCR_Internet_Collection_EUS33.jpg",1)
# a
# # list_t_f = detect_text("images/OCR_Internet_Collection_EUS33.jpg")
# # list_t_f

In [ ]:
import os

list_dir = os.listdir('images')
try:
    list_dir.remove('.ipynb_checkpoints')
except:
    pass
len(list_dir)
dict_temp0 = []
dict_temp = {}
dict_temp1 = {}
for a in list_dir:
    try:
        path = 'images/' + a
        flag, cor,len_edge =  last_check(path)
        dict_temp0.append(flag)
        # cor, len_edge = detect_cordinate2('original-image.png')
        dict_temp[a] = cor
        dict_temp1[a] = len_edge
    except:
        print(a)

Untitled.ipynb


In [238]:
import pandas as pd
df = pd.DataFrame(columns = ["img", "coordinate", "edge_len"])
df["img"] = dict_temp.keys()
df["flag"] = dict_temp0
df["coordinate"] = dict_temp.values()
df["edge_len"] = dict_temp1.values()

In [239]:
df[df["flag"] == False]["coordinate"][23]

({1: (1203788.667, 606386.803),
  2: (1203776.856, 606382.475),
  3: (1203773.459, 606373.517),
  4: (1203785.359, 606377.911)},
 [14.78, 5.5, 14.9, 5.5])

In [240]:
df

,img,coordinate,edge_len,flag
0,test-300.png,"({1: (1201283.78, 604990.09), 2: (1201286.67, ...",1.0,True
1,OCR_Internet_Collection_EUS18.jpg,"({1: (1204311.54, 599692.47), 2: (1204250.1, 5...",1.0,False
2,OCR_Internet_Collection_EUS06.jpg,"({1: (1201283.78, 604990.09), 2: (1201286.67, ...",1.0,True
3,OCR_Internet_Collection_EUS22.jpg,"({1: (1182029.662, 603722.679), 2: (1182031.94...",1.4,True
4,STM_topography_resized.png,"({1: (1201283.78, 604990.09), 2: (1201286.67, ...",1.1,True
5,OCR_Internet_Collection_EUS08.jpg,"({1: (1194022.31, 601392.82), 2: (1194016.0, 6...",1.5,True
6,OCR_Internet_Collection_EUS32.jpg,"({1: (1198267.83, 596965.38), 2: (1198281.43, ...",1.0,True
7,OCR_Internet_Collection_EUS12.jpg,"({1: (1181471.6, 595704.35), 2: (1181458.24, 5...",1.0,True
8,OCR_Internet_Collection_EUS13.jpg,"({1: (1180831.61, 592118.2), 2: (1180832.91, 5...",1.1,True
9,OCR_Internet_Collection_EUS07.jpg,"({1: (1225890.38, 582654.01), 2: (1225886.56, ...",1.0,True
